In [96]:
!pip install tensorflow

### Part 1 : Data Proprocessing

In [97]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [98]:
tf.__version__

'2.19.0'

In [99]:
dataset = pd.read_csv("./Churn_Modelling.csv")
X = dataset.iloc[ : , 3:-1].values
y = dataset.iloc[ :, -1].values

>Encoding the genden in the dataset -- DEPENDENT

In [100]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[: , 2] = le.fit_transform(X[: , 2]) # making the gender to 0 and 1 code format

> Encoding the country - Independent variables ( ONT HOT ENCODING )

In [101]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder' , OneHotEncoder() , [1]) ] ,remainder='passthrough')
X  = np.array(ct.fit_transform(X))

In [102]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.25, random_state=0)

> Feature Scaling -- MOST IMPORTANT in Deep learning

In [103]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Part 2 : Building ANN

> Making Ann

In [104]:
ann = tf.keras.models.Sequential()

> Adding the input layer and the first hidden layer

In [105]:
ann.add(tf.keras.layers.Dense(units = 6 , activation='relu'))
# add() is the method that is used to add the layer in the ANN 
# activation function for the hidden layer is rectifier function

> Adding the second hidden layer

In [106]:
ann.add(tf.keras.layers.Dense(units = 6 , activation='relu'))

> Adding the Output layer

Here we have output in the data in 0/1 format that's why unit is 1
if we had the output data in 3 or more format then we have to increase the number of the units

In [107]:
ann.add(tf.keras.layers.Dense(units = 1 , activation='sigmoid'))

### Part 3 : Training ANN

> Compiling the ANN

In [108]:
ann.compile(optimizer= 'adam' , loss= 'binary_crossentropy', metrics= ['accuracy'])

> Training the ANN

In [109]:
ann.fit(X_train, Y_train, batch_size = 42, epochs = 120)
# batch_size = 32 : The model will use 32 training samples per gradient update (mini-batch gradient descent)
# epochs : 	The entire training dataset will be passed through the model 100 times

Epoch 1/120
179/179 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7073 - loss: 0.6105
Epoch 2/120
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7946 - loss: 0.4582
Epoch 3/120
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8028 - loss: 0.4377  
Epoch 4/120
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8014 - loss: 0.4342  
Epoch 5/120
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7987 - loss: 0.4238  
Epoch 6/120
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8043 - loss: 0.4219
Epoch 7/120
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8196 - loss: 0.4082
Epoch 8/120
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8311 - loss: 0.3939
Epoch 9/120
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8350 - loss: 0.3939
Epoch 10/120
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8425 - loss: 0.3767
Epoch 11/120
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8495 - loss: 0.3670
Epoch 12/120
179/179 ━━━━━━━━━━━━━━

### Part 4 : Making the predictions and evaluating the model

> Single out for one customer


Geography: France
Credit Score: 600
Gender: Male
Age: 40 years old
Tenure: 3 years
Balance: \$ 60000
Number of Products: 2
Does this customer have a credit card? Yes
Is this customer an Active Member: Yes
Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

In [110]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)
# use the  [[]] --> 2D array
# france  --> 1 , 0, 0 after the one hot encoding
# feature --> scaling needed to be done
# () > 0.5 --> activation function -- values and vary according to the dataset

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

> Predict the test set results

In [111]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), Y_test.reshape(len(Y_test),1)),1))

79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


> Confusion Matrix

In [112]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(Y_test, y_pred)
print(cm)
accuracy_score(Y_test, y_pred)

[[1892   99]
 [ 249  260]]


0.8608